# Yomitoku Pro Document Analyzer in SageMaker

## はじめに
aws-cliを利用して、aws configureの設定を行ってください。ユーザのアクセスキーを生成し、aws cli credentialsの紐付けを行います。

### aws-cliのインストール

** Mac OS **
```
brew update
brew install awscli
```

** Linux **
```
sudo apt-get update
sudo apt-get install -y awscli
```

** Windows **
```
winget install --id Amazon.AWSCLI -e
```

### アクセスキーの発行
#### 🧩 ステップ1：AWS マネジメントコンソールにログイン

1. https://console.aws.amazon.com/にアクセス\
2. 右上のアカウント名を確認
→ ルートユーザーではなく IAMユーザー（例：taro.tanaka@mlism.co.jp）でログインしていることを確認

#### ⚙️ ステップ2：IAM 管理画面を開く

1. サービス検索バーで「IAM」と入力し、選択
2. 左メニューから「ユーザー (Users)」をクリック
3. 対象のユーザー名（例：taro.tanaka）をクリック

#### 🔑 ステップ3：アクセスキーを発行する

1. 「セキュリティ認証情報 (Security credentials)」タブを開く\
2. 「アクセスキー」セクションまでスクロール
3. 「アクセスキーを作成 (Create access key)」をクリック


### AWS Configureの設定
発行したアクセスキーをaws cli credentialsの紐付けを行います。

```
aws configure
```

以下のような入力を求められます。発行したアクセスキーの情報を入力します。
```
AWS Access Key ID [None]: AKIA***************
AWS Secret Access Key [None]: wJalrXUtnFEMI/K7MDENG/bPxRfiCY**********
Default region name [None]: ap-northeast-1
Default output format [None]: json
```

## SageMakerエンドポイントの作成
事前に **CloudFormation** または **SageMaker** コンソールを使って、このモデルのエンドポイントを作成します。

**必要な権限**: AWS ユーザー/ロールに SageMaker FullAccess 権限があることを確認してください。

エンドポイントの作成方法や[URL]を参照してください

### 事前セットアップ
必要なライブラリをインストールします。

In [1]:
import json
import boto3
from botocore.exceptions import ClientError
from pprint import pprint

### エンドポイントとのセッションを確立
以下の情報を入力してください。
- `ENDPOINT_NAME`: SageMakerのエンドポイント名を指定します。
- `AWS_REGION`: エンドポイントを立ち上げたAWS REGIONを指定します。

AWSユーザーの認証にMFAを利用する場合は以下の情報を入力してください。
MFAを利用しない場合は変数はNoneのまま変更は必要ありません。

- `MFA_SEAIAL`: arn:aws:iam::{AWS Account ID}:mfa/{MFA Device Name}
- `MFA_TOKEN`: MFA Token Code

In [2]:
ENDPOINT_NAME = "my-endpoint"
AWS_REGION = "ap-northeast-1"

# Optional: MFA settings
MFA_SERIAL = "arn:aws:iam::025897765203:mfa/iphone"
MFA_TOKEN = None

def with_mfa_session(base_sess: boto3.Session, mfa_serial: str, token: str) -> boto3.Session:
    """
    Create a session with MFA authentication.

    Args:
        base_sess: Base boto3 session (default creds / profile)
        mfa_serial: MFA device ARN (e.g. arn:aws:iam::...:mfa/xxxx)
        token: 6-digit MFA code
    """
    sts = base_sess.client("sts")
    resp = sts.get_session_token(
        SerialNumber=mfa_serial,
        TokenCode=token,
        DurationSeconds=43200  # 12h (受账号上限影响)
    )
    c = resp["Credentials"]
    return boto3.Session(
        region_name=base_sess.region_name,
        aws_access_key_id=c["AccessKeyId"],
        aws_secret_access_key=c["SecretAccessKey"],
        aws_session_token=c["SessionToken"],
    )

base = boto3.Session(region_name=AWS_REGION)

# -------- Build session (default or MFA) --------
if MFA_SERIAL:
    token = MFA_TOKEN or input("Enter MFA code: ").strip()
    try:
        session = with_mfa_session(base, MFA_SERIAL, token)
        print("✅ Using MFA session credentials")
    except ClientError as e:
        raise SystemExit(f"❌ MFA failed: {e}")
else:
    session = base
    print("ℹ️ Using default AWS credentials/profile")

# -------- Clients --------
sagemaker_runtime = session.client('sagemaker-runtime')
sagemaker         = session.client('sagemaker')

# -------- Optional: verify endpoint status --------
try:
    status = sagemaker.describe_endpoint(EndpointName=ENDPOINT_NAME)["EndpointStatus"]
    print(f"Endpoint status: {status}")
except ClientError as e:
    print(f"Error checking endpoint status: {e}")
    print("Make sure your endpoint name is correct and you have proper permissions.")

✅ Using MFA session credentials
Endpoint status: InService


## エンドポイントの呼び出し

この API は、ドキュメントに対して, AI-OCR処理及びレイアウト解析を実行し、読み取り文字列情報やバウンディングボックス（領域座標）、信頼スコア、階層構造化されたテキストブロックを含む構造化データを返します。

レスポンスのスキーマ構造やAPIの仕様の詳細ついては以下を参照してください。
https://mlism-marketplace-documents.s3.ap-northeast-1.amazonaws.com/yomitoku-pro-document-analyzer-v1-3-2.html

** Note **

- 大きなファイルについて：大容量の PDF ファイルを処理する場合は、タイムアウトになる可能性もあるため、可能であれば小さなバッチに分割して処理することを推奨します。

- エラーハンドリング： 本番環境のコードでは、必ず適切なエラーハンドリングを組み込んでください。APIのエラーに関しては仕様書を参照してください。

- パフォーマンス：GPU インスタンスを種類によって、ドキュメントの処理速度が異なります。よりハイエンドのGPUインスタンスを利用することで、より高速に処理可能です。

** Supported File Types **

Yomitoku-Pro SageMakerは以下のファイルタイプをサポートしてます。リクエスト時に`ContentType`を指定する必要があります。

- **image/jpeg** - JPEG images
- **image/png** - PNG images  
- **image/tiff** - TIFF images
- **application/pdf** - PDF files

### Image Analysis (PNG)

In [3]:
FILE_PATH = "sample/image.png"
CONTENT_TYPE = "image/png"


with open(FILE_PATH, "rb") as f:
    response = sagemaker_runtime.invoke_endpoint(
        EndpointName=ENDPOINT_NAME,
        ContentType=CONTENT_TYPE,
        Body=f.read(),
    )

body_bytes = response["Body"].read()
result = json.loads(body_bytes)

pprint(f"{result['result']}"[:500])

# Save full result to a JSON file
with open("result.json", "w") as f:
    json.dump(result, f, indent=2, ensure_ascii=False)

=== Image Analysis Results ===
("[{'preprocess': {'angle': 0.0, 'angle_score': 1.0}, 'paragraphs': [{'box': "
 "[1, 75, 747, 104], 'contents': 'TELEWORK\\nLEWORK', 'direction': "
 "'horizontal', 'order': 0, 'role': 'page_header', 'indent_level': None}, "
 "{'box': [907, 76, 2282, 105], 'contents': 'ORK\\nLEWORK TELEWORK', "
 "'direction': 'horizontal', 'order': 1, 'role': 'page_header', "
 "'indent_level': None}, {'box': [2367, 78, 3298, 106], 'contents': "
 "'テレワークのさらなる普及・定着に向け「テレワーク月間」を実施します!\\nTELEWORK', 'direction': 'horizontal', "
 "'order': 2, 'role': ")


### PDF Analysis
PDFは複数ページを同時に送信すると、タイムアウトやメモリオーバフローが発生する可能性があるため、ページごとに分割し、リクエストすることを推奨します。

以下のコードはPDFをPNG形式のバイナリデータに変換し、ページ単位でエンドポイントにリクエストを送信し、レスポンスを結合し、保存しています。

In [4]:
import time
from yomitoku_client.utils import load_pdf_to_bytes

FILE_PATH = "./sample/image.pdf"
CONTENT_TYPE = "image/png"
DPI = 200

pdf_pytes = load_pdf_to_bytes(FILE_PATH, dpi=DPI)

results = None
for page_num, page_bytes in enumerate(pdf_pytes, start=1):
    print(f"--- Processing page {page_num} ---")
    start  = time.time()
    response = sagemaker_runtime.invoke_endpoint(
        EndpointName=ENDPOINT_NAME,
        ContentType=CONTENT_TYPE,
        Body=page_bytes,
    )

    body_bytes = response["Body"].read()

    end = time.time()
    pprint(f"Processing time: {end - start:.2f} seconds")
    pprint(f"Page {page_num} result preview:")
    pprint(f"{json.loads(body_bytes)['result']}"[:500])

    if results is None:
        results = json.loads(body_bytes)
    else:
        page_result = json.loads(body_bytes)
        results["result"] += page_result["result"]

# Save full result to a JSON file
with open("result_pdf.json", "w") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

--- Processing page 1 ---
'Processing time: 1.50 seconds'
'Page 1 result preview:'
("[{'preprocess': {'angle': 0.0, 'angle_score': 0.9999990463256836}, "
 "'paragraphs': [{'box': [234, 74, 1424, 128], 'contents': "
 "'もっと知りたい!現在·未来のくらしと生活の情報誌', 'direction': 'horizontal', 'order': 0, 'role': "
 "None, 'indent_level': None}, {'box': [248, 150, 1377, 452], 'contents': "
 "'総務省', 'direction': 'horizontal', 'order': 1, 'role': 'section_headings', "
 "'indent_level': None}, {'box': [81, 486, 313, 521], 'contents': '2024 "
 "年11月号', 'direction': 'horizontal', 'order': 2, 'role': None, 'indent_level': "
 'None}, {')
--- Processing page 2 ---
'Processing time: 4.38 seconds'
'Page 2 result preview:'
("[{'preprocess': {'angle': 0.0, 'angle_score': 1.0}, 'paragraphs': [{'box': "
 "[0, 76, 745, 106], 'contents': 'TELEWORK\\nWORK', 'direction': 'horizontal', "
 "'order': 0, 'role': 'page_header', 'indent_level': None}, {'box': [910, 77, "
 "2289, 106], 'contents': 'RK\\nWORK TELEWORK', 'direction': 

### 手書き画像の処理
YomiToku-Pro SageMakerは手書き文字の認識をサポートしています。

In [6]:
FILE_PATH = "sample/tegaki.jpg"
CONTENT_TYPE = "image/jpeg"

with open(FILE_PATH, "rb") as f:
    start = time.time()
    response = sagemaker_runtime.invoke_endpoint(
        EndpointName=ENDPOINT_NAME,
        ContentType=CONTENT_TYPE,
        Body=f.read(),
    )
    end = time.time()
    pprint(f"Processing time: {end - start:.2f} seconds")

body_bytes = response["Body"].read()
result = json.loads(body_bytes)

pprint(f"{result['result']}"[:500])

# Save full result to a JSON file
with open("tegaki.json", "w") as f:
    json.dump(result, f, indent=2, ensure_ascii=False)

'Processing time: 1.93 seconds'
("[{'preprocess': {'angle': 0.0, 'angle_score': 0.9999984502792358}, "
 "'paragraphs': [{'box': [341, 240, 874, 373], 'contents': '日本人移民', "
 "'direction': 'horizontal', 'order': 0, 'role': 'section_headings', "
 "'indent_level': None}, {'box': [2712, 32, 2803, 103], 'contents': 'No', "
 "'direction': 'horizontal', 'order': 1, 'role': None, 'indent_level': None}, "
 "{'box': [2709, 144, 2853, 216], 'contents': 'Date', 'direction': "
 "'horizontal', 'order': 2, 'role': None, 'indent_level': None}, {'box': [441, "
 '389, 3035,')


解析結果はYomiToku-Clientライブラリを利用することで以下の処理が実現可能です。
- 様々な形式（CSV、Markdown、HTML、JSON）に変換
- 変換したデータの解析結果を可視化

これらの方法については`notebooks/yomitoku-client-parser.ipynb`で解説します。